In [1]:
import requests
import pandas as pd
import json

# Sample Raw json data from Louisville Metro Open Data API

### From:
https://services1.arcgis.com/79kfd2K6fskCAkyg/arcgis/rest/services/Louisville_Metro_KY_Expenditures_Data_For_Fiscal_Year_2022/FeatureServer/0/query?outFields=*&where=1%3D1&f=geojson

### Show sample data from API in json format 


In [2]:
response = requests.get('https://services1.arcgis.com/79kfd2K6fskCAkyg/arcgis/rest/services/Louisville_Metro_KY_Expenditures_Data_For_Fiscal_Year_2022/FeatureServer/0/query?outFields=*&where=1%3D1&f=geojson')
raw_data = response.json()
print(type(raw_data))
print('----------')
formated_response = json.dumps(raw_data, indent=4)
print(formated_response[:1300])

<class 'dict'>
----------
{
    "type": "FeatureCollection",
    "properties": {
        "exceededTransferLimit": true
    },
    "features": [
        {
            "type": "Feature",
            "id": 1,
            "geometry": null,
            "properties": {
                "Fiscal_Year": 2022,
                "Budget_Type": "Metro Government Capital",
                "Agency_Name": "Public Works & Assets Department",
                "Sub_Agency_Name": "D11 New Sidewalk Stony Brook Dr",
                "DepartmentName": null,
                "Sub_DepartmentName": null,
                "Category": "Equipment/Capital Outlay",
                "Sub_Category": "Right of Way Land",
                "Stimulus_Type": null,
                "Funding_Source": "Federally Funded",
                "Vendor_Name": "PATRICK W LEIBY",
                "InvoiceID": 2439134,
                "InvoiceDt": 1649304000000,
                "InvoiceAmt": 1865,
                "DistributionAmt": 1492,
        

# Create DataFrame
## Clean up 

- limit DataFrame to first 100 of 1000 JSON object literals
- Capture inner dictionary (properties) from features object literals
    - NOT utilizing Pandas json_normalize()
- Create list of "properties" object literals
- Create DataFrame from list of "properties" object literals
- Drop columns not needed


In [3]:
response = requests.get('https://services1.arcgis.com/79kfd2K6fskCAkyg/arcgis/rest/services/Louisville_Metro_KY_Expenditures_Data_For_Fiscal_Year_2022/FeatureServer/0/query?outFields=*&where=1%3D1&f=geojson')
data1 = response.json()
data2 = pd.DataFrame.from_dict(data1['features'][0:100])
data3 = pd.DataFrame.from_dict(data2["properties"])
data4 = data3.to_dict('list')
data5 = pd.DataFrame.from_dict(data4["properties"])
data5 = data5.drop(columns=['Budget_Type','Agency_Name','Sub_Agency_Name','DepartmentName','Sub_DepartmentName','Category','Sub_Category','Stimulus_Type','Funding_Source','InvoiceDt','DistributionAmt','CheckDt','CheckVoidDt','ObjectId'])
#data5.to_excel("data5.xlsx") 
print(data5)

    Fiscal_Year                       Vendor_Name  InvoiceID  InvoiceAmt  \
0          2022                   PATRICK W LEIBY    2439134      1865.0   
1          2022  HALL CONTRACTING OF KENTUCKY INC    2347858     10500.0   
2          2022  HALL CONTRACTING OF KENTUCKY INC    2347858     10500.0   
3          2022  HALL CONTRACTING OF KENTUCKY INC    2347858     10500.0   
4          2022                        S WALK INC    2333017      2500.0   
..          ...                               ...        ...         ...   
95         2022               HDR ENGINEERING INC    2452292      5684.4   
96         2022               HDR ENGINEERING INC    2452292      5684.4   
97         2022               HDR ENGINEERING INC    2460107      5684.4   
98         2022  HALL CONTRACTING OF KENTUCKY INC    2343646     11333.7   
99         2022  HALL CONTRACTING OF KENTUCKY INC    2472493      6645.0   

    CheckID    CheckAmt  
0   1204645     1865.00  
1   1170537    26170.00  
2   11705

### Show DataFrame type and pandas.DataFrame.head (.head()) function 

In [4]:
print(f'Type from response: {type(data5)}')
data5.head()

Type from response: <class 'pandas.core.frame.DataFrame'>


,Fiscal_Year,Vendor_Name,InvoiceID,InvoiceAmt,CheckID,CheckAmt
0,2022,PATRICK W LEIBY,2439134,1865.0,1204645,1865.0
1,2022,HALL CONTRACTING OF KENTUCKY INC,2347858,10500.0,1170537,26170.0
2,2022,HALL CONTRACTING OF KENTUCKY INC,2347858,10500.0,1170537,26170.0
3,2022,HALL CONTRACTING OF KENTUCKY INC,2347858,10500.0,1170537,26170.0
4,2022,S WALK INC,2333017,2500.0,1162344,2500.0


### Show descriptive statistics with pandas.DataFrame.describe (.describe)

In [5]:
data5.describe()

,Fiscal_Year,InvoiceID,InvoiceAmt,CheckID,CheckAmt
count,100.0,1.000000e+02,100.000000,1.000000e+02,1.000000e+02
mean,2022.0,2.405954e+06,13595.520400,1.191914e+06,2.078698e+05
std,0.0,4.731274e+04,32866.440737,1.885787e+04,4.515136e+05
min,2022.0,2.324045e+06,130.000000,1.159148e+06,1.300000e+02
25%,2022.0,2.366662e+06,2482.500000,1.177060e+06,9.895080e+03
50%,2022.0,2.404414e+06,5684.400000,1.191356e+06,5.538786e+04
75%,2022.0,2.449355e+06,10270.080000,1.209637e+06,1.019718e+05
max,2022.0,2.483291e+06,305015.160000,1.221816e+06,1.712309e+06


### Show tuple representing the dimensionality of the DataFrame.  pandas.DataFrame.shape(.shape)

In [6]:
data5.shape

(100, 6)

### Using DataFrame.mean() to get "CheckAmt" column mean

In [7]:
chk_amount_avg = data5['CheckAmt'].mean()
print(chk_amount_avg)

207869.7743


### Using DataFrame.sum() to get CheckAmt column sum

In [8]:
chk_amount_sum = data5['CheckAmt'].sum()
print(chk_amount_sum)

20786977.43


### Find the maximum value of a column (CheckAmt) and to return its corresponding row values

In [9]:
col = "CheckAmt"
max_x = data5.loc[data5[col].idxmax()]
print(max_x)

Fiscal_Year                                2022
Vendor_Name    HALL CONTRACTING OF KENTUCKY INC
InvoiceID                               2324045
InvoiceAmt                             26737.51
CheckID                                 1159148
CheckAmt                             1712308.66
Name: 30, dtype: object


### Pull subset of vendor(TEK SYSTEMS) from Vendor_name, using loc also .query

In [10]:
data6 = data5.loc[data5['Vendor_Name'] == 'TEK SYSTEMS']
data6 = data6.reset_index(drop=True)
print(data6)

   Fiscal_Year  Vendor_Name  InvoiceID  InvoiceAmt  CheckID   CheckAmt
0         2022  TEK SYSTEMS    2402689     7280.00  1190936  115432.00
1         2022  TEK SYSTEMS    2403602     7280.00  1190936  115432.00
2         2022  TEK SYSTEMS    2403610     7280.00  1190936  115432.00
3         2022  TEK SYSTEMS    2403621     7280.00  1190936  115432.00
4         2022  TEK SYSTEMS    2382444     3555.23  1182032    3555.23


In [11]:
data7 = data5.query("Vendor_Name == 'TEK SYSTEMS'")
print(data7)

    Fiscal_Year  Vendor_Name  InvoiceID  InvoiceAmt  CheckID   CheckAmt
37         2022  TEK SYSTEMS    2402689     7280.00  1190936  115432.00
56         2022  TEK SYSTEMS    2403602     7280.00  1190936  115432.00
60         2022  TEK SYSTEMS    2403610     7280.00  1190936  115432.00
70         2022  TEK SYSTEMS    2403621     7280.00  1190936  115432.00
76         2022  TEK SYSTEMS    2382444     3555.23  1182032    3555.23


### Show first 4 rows of DataFrame

In [12]:
data5.iloc[:4]

,Fiscal_Year,Vendor_Name,InvoiceID,InvoiceAmt,CheckID,CheckAmt
0,2022,PATRICK W LEIBY,2439134,1865.0,1204645,1865.0
1,2022,HALL CONTRACTING OF KENTUCKY INC,2347858,10500.0,1170537,26170.0
2,2022,HALL CONTRACTING OF KENTUCKY INC,2347858,10500.0,1170537,26170.0
3,2022,HALL CONTRACTING OF KENTUCKY INC,2347858,10500.0,1170537,26170.0
